In [ ]:
import os

import numpy as np
from keras.optimizers import Adam
from keras.callbacks import Callback, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from PIL import Image

from unet import get_unet, jaccard_coef_int, jaccard_coef_loss

## load the model

In [ ]:
os.environ["CUDA_VISIBILE_DEVICES"] = "2"

In [ ]:
model = get_unet(3, 512, 512)

In [ ]:
adam = Adam(lr=1e-3)
model.compile(adam, loss=jaccard_coef_loss, metrics=['binary_crossentropy', jaccard_coef_int])

In [ ]:
model.summary()

In [ ]:
model.output

In [ ]:
model.outputs

## load data and define generators

In [ ]:
import glob
import cv2
import numpy as np
import random
import math
import json
SEED = 448
import matplotlib.pyplot as plt

In [ ]:
masks = glob.glob('/root/data/gopro/new_labels/*semantic.png')

In [ ]:
len(masks)

In [ ]:
0.8*479

In [ ]:
random.seed(SEED)
random.shuffle(masks)
train = masks[:383]
val = masks[383:]

In [ ]:
masks[0]

In [ ]:
np.array(Image.open(masks[0])).sum(axis=2)

In [ ]:
batch_size = 8
steps_per_epoch = len(train) // batch_size
steps_per_epoch_val = len(val) // batch_size
input_shape = (512, 512, 3)

In [ ]:
train[0]

In [ ]:
test_mask = np.array(Image.open(train[0]))

In [ ]:
plt.imshow(test_mask)

In [ ]:
red, green, blue = test_mask[:,:,0], test_mask[:,:,1], test_mask[:,:,2]
blue_mask = (red == 0) & (green == 0) & (blue == 255)
pink_mask = (red == 255) & (green == 105) & (blue == 180)

In [ ]:
def generator(masks, steps_per_epoch, BATCH_SIZE, input_shape):
    i = 0
    img_size = input_shape[0]
    while True:
        x_batch = np.empty((BATCH_SIZE, input_shape[0], input_shape[1], input_shape[2]), dtype=np.uint8)
        y_batch = np.empty((BATCH_SIZE, input_shape[0], input_shape[1], 2), dtype=np.uint8)
        for (ind, j) in enumerate(range(i*BATCH_SIZE, (i+1)*BATCH_SIZE)):
            mask_path = masks[j]
            img_name = os.path.basename(mask_path).split('.')[0] + '.png'
            img_path = '/root/data/gopro/selected_frames/' + img_name
            # masks_name = os.path.basename(img_path).split('.')[0]
            # mask_path = '/root/data/sflab_ground_truth/v2_071218/trunc_semantic_masks/{}.npy'.format(masks_name)
            x_batch[ind,...] = np.array(Image.open(img_path).resize((input_shape[0], input_shape[1])))
            
            mask_img = np.array(Image.open(mask_path))
            mask0 = np.zeros((1080, 1920))
            mask1 = np.zeros((1080, 1920))
            
            red, green, blue = mask_img[:,:,0], mask_img[:,:,1], mask_img[:,:,2]
            blue_mask = (red == 0) & (green == 0) & (blue == 255)
            pink_mask = (red == 255) & (green == 105) & (blue == 180)
            
            
            
            mask0[blue_mask] = 1
            mask1[pink_mask] =1
            y_batch[ind,...,0] = cv2.resize(mask0, (input_shape[0], input_shape[1]))
            y_batch[ind,...,1] = cv2.resize(mask1, (input_shape[0], input_shape[1]))
        i += 1
        if i >= steps_per_epoch:
            i = 0
        yield x_batch, y_batch

In [ ]:
train_generator = generator(train, steps_per_epoch, batch_size, input_shape)
val_generator = generator(val, steps_per_epoch_val, batch_size, input_shape)

In [ ]:
X, Y = train_generator.next()

## train

In [ ]:
# learning rate schedule
def step_decay(epoch):
    initial_lrate = 1e-3
    drop = 0.5
    epochs_drop = 10.0
    if epoch >= 30:
        fake_epoch = epoch - 20 
        lrate = initial_lrate * math.pow(drop, math.floor((1+fake_epoch)/epochs_drop))
    else:
        lrate = initial_lrate
    print('lr {}'.format(lrate))
    return lrate
lr_scheduler = LearningRateScheduler(step_decay)

In [ ]:
# create history callback
class SaveHistory(Callback):
    
    def __init__(self, json_path):
        self.json_path = json_path
    
    def on_train_begin(self, logs=None):
        self.epoch = []
        self.history = {}

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        self.epoch.append(epoch)
        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
        with open(self.json_path, 'w') as f:
            json.dump(self.history, f)

In [ ]:
class MAP_eval(Callback):
    def __init__(self, validation_data):
        self.validation_data = validation_data
        self.maps = []

    def eval_map(self):
        x_val, y_true = self.validation_data
        y_pred = self.model.predict(x_val)
        y_pred = list(np.squeeze(y_pred))
        zipped = zip(y_true, y_pred)
        zipped.sort(key=lambda x:x[1],reverse=True)

        y_true, y_pred = zip(*zipped)
        k_list = [i for i in range(len(y_true)) if int(y_true[i])==1]
        score = 0.
        r = np.sum(y_true).astype(np.int64)
        for k in k_list:
            Yk = np.sum(y_true[:k+1])
            score += Yk/(k+1)
        score/=r
        return score

    def on_epoch_end(self, epoch, logs={}):
        score = self.eval_map()
        print "MAP for epoch %d is %f"%(epoch, score)
        self.maps.append(score)

In [ ]:
saveh = SaveHistory('./segmentation_history.json')

In [ ]:
# save model
filepath = os.path.join('/root/data/models/gopro/segmentation/', 'segmentation_{epoch:02d}.h5')
checkpoint = ModelCheckpoint(filepath, 
                             monitor='val_jaccard_coef_int', 
                             save_best_only=True, 
                             mode='max')

In [ ]:
# start training# start 
history = model.fit_generator(
        generator=train_generator,
        steps_per_epoch=steps_per_epoch,
        epochs=50,
        verbose=1,
        callbacks=[lr_scheduler],
        validation_data= val_generator,
        validation_steps= steps_per_epoch_val)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
history = json.load(open('segmentation_history.json'))

In [ ]:
history.keys()

In [ ]:
np.max(history['val_jaccard_coef_int'])

In [ ]:
plt.plot(history['jaccard_coef_int'], label='jaccard_coef_int')
plt.plot(history['val_jaccard_coef_int'], label='val_jaccard_coef_int')
plt.legend()

In [ ]:
plt.plot(history['loss'], label='train_loss')
plt.plot(history['val_loss'], label='val_loss')
plt.legend()

# predict

In [ ]:
X, Y = val_generator.next()

In [ ]:
Ypred = model.predict_on_batch(X).squeeze()

In [ ]:
Ypred.shape

In [ ]:
for i in range(8):
    f, ax = plt.subplots(1, 2, figsize=(20, 10))
    ax[0].imshow(X[i,...])
    ax[0].imshow(Y[i,...,0], alpha=0.5)
    ax[0].set_title("background")
    
    ax[1].imshow(X[i,...])
    ax[1].imshow(Y[i,...,1], alpha=0.5)
    ax[1].set_title("foreground")
    plt.show()